In [1]:
import os
from gurobipy import Model, GRB
import matplotlib.pyplot as plt


def read_exams(file_path):
    exams = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.split()
            if len(parts) != 2:
                print(f"Skipping malformed line in exams file: {line.strip()}")
                continue
            exam_id, num_students = map(int, parts)
            exams[exam_id] = num_students
    return exams

def read_time_slots(file_path):
    with open(file_path, 'r') as file:
        return int(file.readline().strip())

def read_students(file_path):
    students = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.split()
            if len(parts) != 2:
                print(f"Skipping malformed line in students file: {line.strip()}")
                continue
            student_id = parts[0]
            exam_id = int(parts[1])
            if student_id not in students:
                students[student_id] = []
            students[student_id].append(exam_id)
    return students

def build_conflicts(students):
    conflicts = {}
    for exams in students.values():
        for i in range(len(exams)):
            for j in range(i + 1, len(exams)):
                e1, e2 = exams[i], exams[j]
                if (e1, e2) not in conflicts and (e2, e1) not in conflicts:
                    conflicts[(e1, e2)] = 0
                conflicts[(e1, e2)] += 1
    return conflicts


In [2]:
def solve_etp(exams, students, time_slots, conflicts):
    model = Model('ETP')
    
    # Variables
    x = model.addVars(exams.keys(), range(1, time_slots + 1), vtype=GRB.BINARY, name='x')
    
    # Constraints
    # Each exam must be scheduled exactly once
    for e in exams.keys():
        model.addConstr(sum(x[e, t] for t in range(1, time_slots + 1)) == 1)
    
    # Conflicting exams cannot be in the same time slot
    for (e1, e2), num_students in conflicts.items():
        for t in range(1, time_slots + 1):
            model.addConstr(x[e1, t] + x[e2, t] <= 1)
    
    # Objective: Minimize the penalty for close conflicts
    penalty = sum(2**(5-i) * num_students * (x[e1, t] * x[e2, t+i])
                  for (e1, e2), num_students in conflicts.items()
                  for t in range(1, time_slots - 5 + 1)
                  for i in range(1, 6))
    model.setObjective(penalty, GRB.MINIMIZE)
    
    # Optimize the model
    model.optimize()
    
    # Check and return results
    if model.Status == GRB.OPTIMAL or model.Status == GRB.TIME_LIMIT:
        solution = {e: t for e in exams.keys() for t in range(1, time_slots + 1) if x[e, t].X > 0.5}
        return solution, model.ObjVal
    else:
        return None, None


In [3]:
def visualize_timetable(solution, time_slots):
    exams_per_slot = {}
    for exam, slot in solution.items():
        if slot not in exams_per_slot:
            exams_per_slot[slot] = []
        exams_per_slot[slot].append(exam)

    fig, ax = plt.subplots()
    ax.set_title('Examination Timetable')
    ax.set_xlabel('Time Slots')
    ax.set_ylabel('Exams')
    
    for slot in range(1, time_slots + 1):
        exams = exams_per_slot.get(slot, [])
        for exam in exams:
            ax.plot(slot, exam, 'ro')
    
    plt.show()


In [4]:
def validate_solution(solution, exams, students, conflicts, time_slots):
    # Validate each exam is scheduled exactly once
    scheduled_exams = set(solution.keys())
    all_exams = set(exams.keys())
    if scheduled_exams != all_exams:
        print("Validation Failed: Some exams are not scheduled exactly once.")
        return False

    # Validate no conflicting exams in the same time slot
    for (e1, e2), _ in conflicts.items():
        if solution[e1] == solution[e2]:
            print(f"Validation Failed: Conflicting exams {e1} and {e2} are scheduled in the same time slot.")
            return False

    print("All constraints are satisfied.")
    return True

def calculate_objective_value(solution, conflicts):
    penalty = 0
    for (e1, e2), num_students in conflicts.items():
        distance = abs(solution[e1] - solution[e2])
        if 1 <= distance <= 5:
            penalty += 2**(5 - distance) * num_students
    return penalty

In [ ]:
def write_output(filename, solution):
    with open(filename, 'w') as file:
        if not solution:
            file.write("UNFEASIBLE\n")
        else:
            for exam, time_slot in sorted(solution.items()):
                file.write(f"{exam} {time_slot}\n")

In [5]:
def main(instance):
    # Define the instance to solve
    exam_file = f"input/{instance}.exm"
    time_slots_file = f"input/{instance}.slo"
    student_file = f"input/{instance}.stu"
    output_file = f"output/{instance}.out"
    
    # Read input files
    try:
        exams = read_exams(exam_file)
        time_slots = read_time_slots(time_slots_file)
        students = read_students(student_file)
    except Exception as e:
        print(f"Error reading input files: {e}")
        return
    
    # Build conflicts
    conflicts = build_conflicts(students)
    
    # Solve ETP
    solution, obj_val = solve_etp(exams, students, time_slots, conflicts)
    
    # Validate solution
    if solution:
        print(f"Optimal solution found with objective value: {obj_val}")
        for exam, time_slot in solution.items():
            print(f"Exam {exam} is scheduled at time slot {time_slot}")

        # Validate constraints
        is_valid = validate_solution(solution, exams, students, conflicts, time_slots)
        
        if is_valid:
            # Calculate and print the objective value
            calculated_obj_val = calculate_objective_value(solution, conflicts)
            print(f"Calculated objective value: {calculated_obj_val}")

            # Visualize the timetable
            visualize_timetable(solution, time_slots)
            
            # Write the output to file
            write_output(output_file, solution)
    else:
        print("No feasible solution found or optimization time limit reached")
        write_output(output_file, None)

# Run the main function with the desired instance
instance_name = "instance01"
main(instance_name)


Skipping malformed line in exams file: 
Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-09


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 99007 rows, 3801 columns and 201453 nonzeros
Model fingerprint: 0xaf9f4a39
Model has 376480 quadratic objective terms
Variable types: 0 continuous, 3801 integer (3801 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 84945 rows and 0 columns (presolve time = 10s) ...
Presolve removed 84945 rows and 0 columns
Presolve time: 9.79s
Presolved: 17455 rows, 7194 columns, 520862 nonzeros
Variable types: 0 continuous, 7194 integer (3857 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.850000e